In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
First understanding the data

In [ ]:
data= pd.read_csv('/kaggle/input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')
data.shape

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
#going by the column names we can drop some of them after creating a copy of original file
data1= data.copy()
data1= data1.drop (['title', 'shipping_option_name', 'merchant_title', 'merchant_name', 'merchant_info_subtitle', 'merchant_id',
                   'merchant_has_profile_picture', 'merchant_profile_picture', 'product_url', 'product_id', 'crawl_month', 
                    'theme', 'currency_buyer','title_orig', 'rating_five_count', 'rating_four_count', 'rating_three_count',
                   'rating_two_count', 'rating_one_count', ], axis=1)

In [ ]:
data1.columns

In [ ]:
#checking missing values
data1[data1==0].count()

In [ ]:
data1.describe()

In [ ]:
#the products are being shipped to various countries
#copied this code from one of Kaggle notebook for FIFA 19 complete player dataset
countries_new= data1['countries_shipped_to'].value_counts()
countries_new= countries_new[countries_new<14].index
data1['countries_shipped_to']= data1['countries_shipped_to'].replace({x:'000' for x in countries_new})

In [ ]:
data1.dtypes

In [ ]:
#converting countries_shipped_to into an integer value
data1['countries_shipped_to'] = pd.to_numeric(data1['countries_shipped_to'])

In [ ]:
#visualizing the top 5 countries where the products are being shipped
sns.countplot(data1.countries_shipped_to)
plt.xticks(rotation=90)

The country codes are not mentioned in the dataset but we can see that the top 5 countries where most of the products are being shipped are 41, 43, 40, 38 and 36

In [ ]:
#Some visulaization of the output i.e. the number of units sold with various input variables will help to understand
#the data in a much better way

In [ ]:
#there is not much affect whether an ad has been run for a product or not
plt.figure(figsize=(10,6))
# make barplot
sns.barplot(x='uses_ad_boosts', y="units_sold", data=data1)
# set labels
plt.xlabel("Ads done", size=15)
plt.ylabel("Number of units sold", size=15)
plt.title("Effect of running ads on the number of units sold for a product", size=18)

In [ ]:
#understanding the relationship between retial price of a product with its unit
sns.relplot(x= 'units_sold', y='retail_price', sizes= (15,200), data=data1)
plt.xlabel('Number of units sold')
plt.ylabel("Retail price of the product")
plt.title('Retail price vs Number of units sold')
plt.xticks(rotation=45)

In [ ]:
#We can clearly observe that product with less price are supposed to sale more

In [ ]:
sns.relplot(x= 'units_sold', y='price', sizes= (15,200), data=data1)
plt.xlabel('Number of units sold')
plt.ylabel("Price of the product")
plt.title('Price vs number of units sold for a product')
plt.xticks(rotation=45)

In [ ]:
#cannot compare for price and retail price with the number of units sold for a product bcoz of uneven distribution of data point

In [ ]:
#the number of units sold for any product increases with the number of iinventories but to a certain limit only
sns.relplot(x= 'units_sold', y='product_variation_inventory', data=data1)
plt.xlabel('Number of units sold')
plt.ylabel("Inventory of the product")
plt.xticks(rotation=45)

In [ ]:
sns.barplot (x= 'badge_local_product', y= 'units_sold', data= data1)
plt.xlabel('Local Product Badge')
plt.ylabel('Number of units sold')
plt.title('Local product vs Number of units sold')
#people are not ready to buy local products

In [ ]:
#as the rating of the product increases number of units sold also increases
sns.relplot(x= 'units_sold', y='rating', data=data1)
plt.xlabel('Number of units sold')
plt.ylabel("Rating of the product")
plt.title('Effect of product rating in its sale')
plt.xticks(rotation=45)

In [ ]:
#as the merchant rating of the product increases number of units sold also increases
sns.relplot(x= 'units_sold', y='merchant_rating', data=data1)
plt.xlabel('Number of units sold')
plt.ylabel("Merchant Rating of the product")
plt.title('Effect of merchant rating on the product sale')
plt.xticks(rotation=45)

In [ ]:
#quality of the product plays an important role in its sale
sns.barplot (x= 'badge_product_quality', y= 'units_sold', data= data1)
plt.xlabel('Product Quality Badge')
plt.ylabel('Number of units sold')
plt.title('Quality of the product vs its sales')

In [ ]:
print('The maximum number of units sold for any product is:', data1['units_sold'].max())
print('The minimum number of units sold for any product is:', data1['units_sold'].min())
print('The average number of units sold for any product is:', data1['units_sold'].mean())
print('The average price for all the product in the month of August 2020 is:', data1.price.mean())
print('The average retail price of all the products in the month of August 2020 is:', data1.price.mean())

In [ ]:
data1.describe()

In [ ]:
#generatiing heatmap
corrmat = data1.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data1[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
#from the heat map we can clearly see that rating count, merchant rating count, merchant rating, 
#product variation inventory, badge product quality, badge fast shipping, badge local product, 
#shippingis express, countries shipped to, price, retail price are the variables which are affecting the output to a much greater extent
#can drop rest of the variables

In [ ]:
data_model= data1.drop(['uses_ad_boosts', 'tags', 'product_color', 'product_variation_size_id', 'shipping_option_price',
                       'inventory_total', 'has_urgency_banner', 'urgency_text', 'origin_country', 'product_picture'], axis=1)

In [ ]:
data_model.shape

In [ ]:
from sklearn import preprocessing 

encoder = preprocessing.LabelEncoder()

for i in data_model.columns:
    if isinstance(data_model[i][0], str):
            data_model[i] = encoder.fit_transform(data_model[i])

In [ ]:
from sklearn.preprocessing import StandardScaler 
  
scalar = StandardScaler() 
  
scalar.fit(data_model) 
scaled_data = scalar.transform(data_model)

In [ ]:
X= data_model.drop('units_sold', axis=1)

In [ ]:
y= data_model.units_sold

In [ ]:
#Using Linear Regression model

In [ ]:
#train test split
from sklearn.model_selection import train_test_split
x_tr, x_ts, y_tr, y_ts = train_test_split (X,y, test_size=0.25, random_state = 1333)

In [ ]:
from sklearn import linear_model
lm = linear_model.LinearRegression()

In [ ]:
model_tr = linear_model.LinearRegression()
model_tr.fit(x_tr, y_tr)
model_tr.score(x_tr, y_tr)

In [ ]:
model_tr.fit(x_ts, y_ts)
model_tr.score(x_ts, y_ts)

In [ ]:
#with Linear Regression model, for predicting the number of units sold for any product the accuracy achieved is 79%